In [3]:
# Import packages
import pandas as pd
import numpy as np
import requests
from scrapy.selector import Selector
from IPython.display import clear_output
import time
import re

#https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b

In [ ]:
# Iterating over search titles, which means that all words must be in the string for the job title
search_titles = ["Sustainability Manager", "Environmental Sustainability Specialist", "Corporate Social Responsibility (CSR) Coordinator", "Sustainable Development Analyst", "Renewable Energy Project Manager", "Climate Change Policy Analyst", "Circular Economy Strategist", "Green Building Consultant", "Sustainability Consultant", "Supply Chain Sustainability Manager"]

In [ ]:
'''
Extract all job URLs for each search_title
'''

# Initialise empty list related to search term
job_URL_list = []
search_title_list = []

# Start timer
start = time.time()

# Iterating over search titles, job types and salary ranges..
for stitle in search_titles:

    # Crawl to the first page of the search results
    path = f"https://au.indeed.com/jobs?as_ttl={stitle.replace(' ','+')}&sr=directhire&radius=0&l=Australia&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch"
    req = requests.get(path)

    # Extract the URLs for extra pages
    extra_page_URL_list = Selector(text=req.text).xpath('//div[@class="pagination"]/a/@href').getall()
    extra_page_URL_list = extra_page_URL_list[:-1] # drop the last URL, due to next button URL
    extra_page_URL_list = ['http://au.indeed.com' + i for i in extra_page_URL_list]

    print(f'Page 1 of {len(extra_page_URL_list)+1}:', end = '\n\n')

    print(f'Searching for all jobs titles that include: {stitle}',end = '\n')

    # Extract all URLs for all the job listings in page 1
    page_job_URL_list = Selector(text=req.text).xpath('//a[@data-tn-element="jobTitle"]/@href').getall()
    print(f"{len(page_job_URL_list)} job URLs extracted from Page 1", end = '\n\n')

    # Append all job URLs
    # Append search titles, job types and salary ranges to match number of job URLs
    job_URL_list += page_job_URL_list
    for i in range(len(page_job_URL_list)):
        search_title_list.append(stitle)


     # To make sure all list lengths are equal
    print('Job URL count:',len(job_URL_list))
    print('Search title count:',len(search_title_list))
    clear_output(wait=True)

    time.sleep(0.1)

    # Iterate through all the additional pages
    page_count = 1
    for extra_page in extra_page_URL_list:
        page_count += 1

        # Crawl to extra page
        req = requests.get(extra_page)
        print(f'Page {page_count} of {len(extra_page_URL_list)+1}:', end = '\n\n')

        print(f'Searching for all jobs titles that include: {stitle}',end = '\n')

        # Extract all URLs for all the job listings in extra pages
        page_job_URL_list = Selector(text=req.text).xpath('//a[@data-tn-element="jobTitle"]/@href').getall()
        print(f"{len(page_job_URL_list)} job URLs extracted from Page {page_count}", end = '\n\n')

        # Append all job URLs
        # Append search titles, job types and salary ranges to match number of job URLs
        job_URL_list += page_job_URL_list
        for i in range(len(page_job_URL_list)):
           search_title_list.append(stitle)

        # To make sure all list lengths are equal
        print('Job URL count:',len(job_URL_list))
        print('Search title count:',len(search_title_list))
        clear_output(wait=True)

        time.sleep(0.1)

job_URL_list = ['http://au.indeed.com' + i for i in job_URL_list]
print(f"Job URL scraping completed: {len(job_URL_list)}", end = '\n')

end = time.time()


#Part Two:https://www.youtube.com/watch?v=eN_3d4JrL_w

In [2]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [3]:
#Setup the query and url
#template= "https://www.indeed.com/jobs?q=sustainability&l=Beloit%2C+WI&from=searchOnHP" -- initial url

def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [5]:
job_positions = ["Sustainability Manager", "Environmental Sustainability Specialist", "Corporate Social Responsibility (CSR) Coordinator", "Sustainable Development Analyst", "Renewable Energy Project Manager", "Climate Change Policy Analyst", "Circular Economy Strategist", "Green Building Consultant", "Sustainability Consultant", "Supply Chain Sustainability Manager"]

location = 'beloit wi'
for position in job_positions:
    url = get_url(position, location)
    print(url)

https://www.indeed.com/jobs?q=Sustainability+Manager&l=beloit+wi
https://www.indeed.com/jobs?q=Environmental+Sustainability+Specialist&l=beloit+wi
https://www.indeed.com/jobs?q=Corporate+Social+Responsibility+(CSR)+Coordinator&l=beloit+wi
https://www.indeed.com/jobs?q=Sustainable+Development+Analyst&l=beloit+wi
https://www.indeed.com/jobs?q=Renewable+Energy+Project+Manager&l=beloit+wi
https://www.indeed.com/jobs?q=Climate+Change+Policy+Analyst&l=beloit+wi
https://www.indeed.com/jobs?q=Circular+Economy+Strategist&l=beloit+wi
https://www.indeed.com/jobs?q=Green+Building+Consultant&l=beloit+wi
https://www.indeed.com/jobs?q=Sustainability+Consultant&l=beloit+wi
https://www.indeed.com/jobs?q=Supply+Chain+Sustainability+Manager&l=beloit+wi


In [22]:
#Extract the html data
#tried to proxies, or headers to avoid this
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)


In [23]:
response

<Response [403]>

In [21]:
response.reason

'Forbidden'

In [38]:
#soup object to navigate the html text editor- I look for is html tag that encloses the highlighted role
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
cards = soup.find_all('div', 'jcs-JobTitle css-jspxzf eu4oa1w0')

<a id="job_0cdb55029eb47b25" data-mobtk="1hpoon26gk6dd811" data-jk="0cdb55029eb47b25" data-hiring-event="false" data-hide-spinner="true" role="button" aria-label="full details of Sustainability Operations Lead" class="jcs-JobTitle css-jspxzf eu4oa1w0" href="/rc/clk?jk=0cdb55029eb47b25&amp;bb=83SAZ_n1rThbM8SMKdR_lZzN2FhjTtYITPKdga7vOwWPWjOzUFza5hfUKx5W0XlwFvsuD36Bp9ufJxRUmGuniUfeNGEKvS9gzCgoXyE98m80SA7rbIvi0w%3D%3D&amp;xkcb=SoBD67M3D45TkXzdZB0LbzkdCdPP&amp;fccid=4c2ad73ae75d8284&amp;vjs=3"><span title="Sustainability Operations Lead" id="jobTitle-0cdb55029eb47b25">Sustainability Operations Lead</span></a>

In [ ]:
#Prototype the model with a single record
card = cards[0]

In [ ]:
job_title = card.h2.a.get('title')

In [ ]:
company = card.find('span', 'company').text.strip()

In [ ]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')

In [ ]:
post_date = card.find('span', 'date').text
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
summary = card.find('div', 'summary').text.strip().replace('\n', ' ')

In [ ]:
# this does not exists for all jobs, so handle the exceptions
salary_tag = card.find('span', 'salaryText')
if salary_tag:
    salary = salary_tag.text.strip()
else:
    salary = ''

In [ ]:
job_url = 'https://www.indeed.com' + card.h2.a.get('href')

In [ ]:
record = (job_title, company, job_location, post_date, today, summary, salary, job_url)

In [ ]:
record

In [ ]:
#Generalize the model with a function
def get_record(card):
    """Extract job data from a single record"""

    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''

    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record

In [ ]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [ ]:
#Get the next page
while True:
    try:
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        break

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')

    for card in cards:
        record = get_record(card)
        records.append(record)

In [ ]:
#Putting it all together
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup


def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url


def get_record(card):
    """Extract job data from a single record"""

    job_title = card.h2.a.get('title')
    company = card.find('span', 'company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    post_date = card.find('span', 'date').text
    today = datetime.today().strftime('%Y-%m-%d')
    summary = card.find('div', 'summary').text.strip().replace('\n', ' ')
    job_url = 'https://www.indeed.com' + card.h2.a.get('href')

    # this does not exists for all jobs, so handle the exceptions
    salary_tag = card.find('span', 'salaryText')
    if salary_tag:
        salary = salary_tag.text.strip()
    else:
        salary = ''

    record = (job_title, company, job_location, post_date, today, summary, salary, job_url)
    return record


def main(position, location):
    """Run the main program routine"""
    records = []
    url = get_url(position, location)

    # extract the job data
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            break

    # save the job data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle', 'Company', 'Location', 'PostDate', 'ExtractDate', 'Summary', 'Salary', 'JobUrl'])
        writer.writerows(records)

In [ ]:
# run the main program
main('senior accountant', 'charlotte nc')

# PrinceIndeed

In [12]:
from httpcore import TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# Setup Chrome options for Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Uncomment the next line to run Chrome in headless mode
# options.add_argument('--headless')

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Setup explicit wait
wait = WebDriverWait(driver, 10)

jobs_data = []
max_pages = 5  # Set a maximum number of pages to scrape
page = 0

while page < max_pages:
    url = f"https://www.indeed.com/jobs?q=media+jobs&l=USA&start={page * 10}"
    driver.get(url)

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
    except TimeoutException:
        print("Timed out waiting for page to load")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    jobs = soup.findAll('div', class_='job_seen_beacon')

    if not jobs:
        print("No more jobs found.")
        break

    for job in jobs:
        data = {}
        try:
            data['Title'] = job.find('h2', class_='jobTitle').text.strip()
        except AttributeError:
            data['Title'] = "Not listed"

        try:
            data['Company'] = job.find('span', class_="css-92r8pb").text.strip()
        except AttributeError:
            data['Company'] = "Not listed"

        try:
            data['Location'] = job.find('div', class_='company_location').text.strip()
        except AttributeError:
            data['Location'] = "Not listed"

        skills_section = job.find('div', class_='jobsearch-SkillsWrapper')
        data['Skills'] = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='jobsearch-SkillTag')]) if skills_section else "Not listed"

        try:
            data['Salary'] = job.find('div', class_='css-1cvo3fd').text.strip()
        except AttributeError:
            data['Salary'] = "Not listed"

        try:
            jobDescriptionText = job.find('div', class_='jobsearch-jobDescriptionText')
            data['Description'] = jobDescriptionText.text.strip() if jobDescriptionText else "Not listed"
        except AttributeError:
            data['Description'] = "Not listed"

        try:
            job_link = job.find('a', class_='jcs-JobTitle')['href']
            data['Apply Link'] = 'https://www.indeed.com' + job_link
        except TypeError:
            data['Apply Link'] = "Not listed"

        jobs_data.append(data)

    print(f"Page {page + 1} completed.")
    page += 1

driver.quit()

df = pd.DataFrame(jobs_data)
df.to_csv('indeed_jobs.csv', index=False)
print("Job extraction complete. Data saved to 'indeed_jobs.csv'.")

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/123.0.6312.58/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5b82033cc993 <unknown>
#1 0x5b82030c7136 <unknown>
#2 0x5b82030fb448 <unknown>
#3 0x5b82030f743d <unknown>
#4 0x5b8203140239 <unknown>
#5 0x5b8203133a73 <unknown>
#6 0x5b8203104c93 <unknown>
#7 0x5b820310565e <unknown>
#8 0x5b820339108b <unknown>
#9 0x5b8203395005 <unknown>
#10 0x5b820337f491 <unknown>
#11 0x5b8203395b92 <unknown>
#12 0x5b82033649ef <unknown>
#13 0x5b82033bbdf8 <unknown>
#14 0x5b82033bbfcb <unknown>
#15 0x5b82033cbae4 <unknown>
#16 0x7a5ff5520ac3 <unknown>


# Part Three

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup as bs
import mysql.connector
import time
from sqlalchemy import create_engine



In [5]:
def connect_insert(data):
#Connect to MySQL database and insert DataFrame data
    try:
        connection = mysql.connector.connect(host='sustainability.cux1s0fa60hj.us-east-2.rds.amazonaws.com',
                                             database='job_scrape',
                                             user='admin',
                                             password='Lenovo2002')

        if connection.is_connected():
            cursor = connection.cursor()
            # Assuming your table has columns: job_id, title, company, location, description
            insert_query = """INSERT INTO job_listings (job_id, title, company, location, skill, job_link, description, website) 
                              VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
            cursor.executemany(insert_query, data)
            connection.commit()
            print(cursor.rowcount, "Records inserted successfully into table")

    except Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


In [6]:
import mysql.connector
from selenium import webdriver


def indeed_scraper(webpage, page_number, wd, cursor):
    # Your scraping logic goes here
    pass

# Establish a connection to MySQL database
mydb = mysql.connector.connect(host='sustainability.cux1s0fa60hj.us-east-2.rds.amazonaws.com',
                                             database='job_scrape',
                                             user='admin',
                                             password='Lenovo2002')
# Create a cursor object
cursor = mydb.cursor()

# Initialize Chrome WebDriver
wd = webdriver.Chrome()

# Call indeed_scraper function with cursor object
indeed_scraper('https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e', 0, wd, cursor)

# Close cursor and database connection
cursor.close()
mydb.close()


In [7]:
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.headless = False # Change to True if you don't want the browser window to open
    driver = webdriver.Chrome(options=options)
    driver.wait = WebDriverWait(driver, 10)
    return driver


In [8]:
def click_see_more(driver):
    attempts = 0
    max_attempts = 3
    
    while attempts < max_attempts:
        try:
            # Wait for the "See more" button to be clickable
            see_more_button = driver.wait.until(
                EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Click to see more description']"))
            )
            
            # Move to the button and click
            ActionChains(driver).move_to_element(see_more_button).click(see_more_button).perform()
            return  # Click succeeded, so return from the function
        except (ElementClickInterceptedException, StaleElementReferenceException, ElementNotInteractableException) as e:
            attempts += 1
            print(f"Attempt {attempts}/{max_attempts} failed: {e}. Retrying...")
            time.sleep(2)  # Wait a bit before retrying
        except TimeoutException:
            return
        except Exception as e:
            return


In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import mysql.connector

def indeed_scraper(webpage, page_number, wd, cursor, mydb):
    next_page = f"{webpage}&start={page_number * 10}"
    print(f"Scraping page: {page_number + 1}")
    wd.get(next_page)

    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("div", class_="jobsearch-SerpJobCard")

    for job in jobs:
        job_id = job["data-jk"]

        select_query = "SELECT JobId FROM sustainability_jobs WHERE JobId = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()

        if result is None:
            job_title = job.find("h2", class_="title").text.strip()
            company_name = job.find("span", class_="company").text.strip()
            location = job.find("span", class_="location").text.strip()
            job_link = "https://www.indeed.com/viewjob?jk=" + job_id

            wd.get(job_link)
            job_src = wd.page_source
            job_soup = bs(job_src, "html.parser")

            job_des_elem = job_soup.find("div", class_="jobsearch-JobComponent-description")
            job_des = job_des_elem.text.strip() if job_des_elem else ""

            insert_query = '''
                INSERT INTO job_listings (JobId, Title, Company, Location, Apply, JobDescription)
                VALUES (%s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
        else:
            print("Skipping duplicate JobId:", job_id)

    print("Page scraped successfully")

    if page_number < 99:
        page_number += 1
        indeed_scraper(webpage, page_number, wd, cursor, mydb)
    else:
        cursor.close()
        mydb.close()
        wd.quit()

# Example usage
wd = webdriver.Chrome()
mydb = mysql.connector.connect(
    host='sustainability.cux1s0fa60hj.us-east-2.rds.amazonaws.com',
                                             database='job_scrape',
                                             user='admin',
                                             password='Lenovo2002')
cursor = mydb.cursor()
indeed_scraper('https://www.indeed.com/jobs?q=sustainability+jobs&l=USA&sc=0kf%3Aexplvl%28ENTRY_LEVEL%29%3B&vjk=93fbcedf8267811e', 0, wd, cursor, mydb)



Scraping page: 1
Page scraped successfully
Scraping page: 2
Page scraped successfully
Scraping page: 3
Page scraped successfully
Scraping page: 4
Page scraped successfully
Scraping page: 5
Page scraped successfully
Scraping page: 6
Page scraped successfully
Scraping page: 7
Page scraped successfully
Scraping page: 8
Page scraped successfully
Scraping page: 9
Page scraped successfully
Scraping page: 10
Page scraped successfully
Scraping page: 11
Page scraped successfully
Scraping page: 12
Page scraped successfully
Scraping page: 13
Page scraped successfully
Scraping page: 14
Page scraped successfully
Scraping page: 15
Page scraped successfully
Scraping page: 16
Page scraped successfully
Scraping page: 17
Page scraped successfully
Scraping page: 18
Page scraped successfully
Scraping page: 19
Page scraped successfully
Scraping page: 20
Page scraped successfully
Scraping page: 21
Page scraped successfully
Scraping page: 22
Page scraped successfully
Scraping page: 23
Page scraped successful

In [43]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import mysql.connector

# Function to connect to MySQL database
def connect_to_mysql():
    return mysql.connector.connect(
        host='database-1.cux1s0fa60hj.us-east-2.rds.amazonaws.com',
        database='job_scrape',
        user='admin',
        password='Lenovo2002'
    )

# Function to scrape job listings from Indeed
def indeed_scraper(webpage, page_number, wd, cursor):
    next_page = f"{webpage}&start={page_number * 10}"
    print(f"Scraping page: {page_number + 1}")
    wd.get(next_page)

    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("div", class_="jobsearch-SerpJobCard")

    for job in jobs:
        job_id = job["data-jk"]

        select_query = "SELECT job_id FROM job_listings WHERE job_id = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()

        if result is None:
            job_title = job.find("h2", class_="title").text.strip()
            company_name = job.find("span", class_="company").text.strip()
            location = job.find("span", class_="location").text.strip()
            job_link = "https://www.indeed.com/viewjob?jk=" + job_id

            driver.get(job_link)
            job_src = driver.page_source
            job_soup = bs(job_src, "html.parser")

            job_des_elem = job_soup.find("div", class_="jobsearch-JobComponent-description")
            job_des = job_des_elem.text.strip() if job_des_elem else ""

            # Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO job_listings (job_id, title, company, location, skill, job_link, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
        else:
            print("Skipping duplicate JobId:", job_id)

    print("Page scraped successfully")

    if page_number < 99:
        page_number += 1
        indeed_scraper(webpage, page_number, wd, cursor)
    else:
        cursor.close()
        driver.quit()

# Establish database connection
mydb = connect_to_mysql()

# Initialize WebDriver
wd = webdriver.Chrome()

# Create cursor object
cursor = mydb.cursor()

# Call indeed_scraper function
indeed_scraper('https://www.indeed.com/jobs?q=sustainability&l=United+States', 0, wd, cursor)

# Close database connection
mydb.close()


Scraping page: 1
Page scraped successfully
Scraping page: 2
Page scraped successfully
Scraping page: 3
Page scraped successfully
Scraping page: 4
Page scraped successfully
Scraping page: 5
Page scraped successfully
Scraping page: 6
Page scraped successfully
Scraping page: 7
Page scraped successfully
Scraping page: 8
Page scraped successfully
Scraping page: 9
Page scraped successfully
Scraping page: 10
Page scraped successfully
Scraping page: 11
Page scraped successfully
Scraping page: 12
Page scraped successfully
Scraping page: 13
Page scraped successfully
Scraping page: 14
Page scraped successfully
Scraping page: 15
Page scraped successfully
Scraping page: 16
Page scraped successfully
Scraping page: 17
Page scraped successfully
Scraping page: 18
Page scraped successfully
Scraping page: 19
Page scraped successfully
Scraping page: 20
Page scraped successfully
Scraping page: 21
Page scraped successfully
Scraping page: 22
Page scraped successfully
Scraping page: 23
Page scraped successful

In [47]:
import mysql.connector
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Function to connect to MySQL database
def connect_to_mysql():
    return mysql.connector.connect(
        host='database-1.cux1s0fa60hj.us-east-2.rds.amazonaws.com',
        database='job_scrape',
        user='admin',
        password='Lenovo2002'
    )

def indeed_scraper(webpage, page_number, wd, cursor):  
    max_pages = 5  # Set a maximum number of pages to scrape
    page = 0

    while page < max_pages:
        url = f"{webpage}&start={page * 10}"
        wd.get(url)

        try:
            WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
        except TimeoutException:
            print("Timed out waiting for page to load")
            break

        soup = BeautifulSoup(wd.page_source, 'html.parser')
        jobs = soup.find_all('div', class_='job_seen_beacon')

        if not jobs:
            print("No more jobs found.")
            break

        for job in jobs:
            data = {}
            try:
                job_id = job['data-jk']
            except KeyError:
                job_id = "Not listed"

            # Extracting job details
            job_title = job.find('span', id=f'jobTitle-{job_id}').text.strip()
            company_name = job.find('span', class_='css-92r8pb').text.strip()
            location = job.find('div', class_='css-1p0sjhy').text.strip()
            skills_section = job.find('div', class_='css-1vjbk9u')
            skill = ', '.join([skill.text.strip() for skill in skills_section.find_all('span', class_='css-g6kqeb')]) if skills_section else "Not listed"
            salary = job.find('div', class_='css-1cvo3fd').text.strip()  # Extract text and then process it further to get salary value
            job_description = job.find('div', class_='css-9446fg').text.strip()  # Extract text from unordered list items
            job_link = 'https://www.indeed.com' + job.find('a', class_='jcs-JobTitle')['href']

            # Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO job_listings (job_id, title, company, location, skill, job_link, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, skill, job_link, job_description)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
            
        print(f"Page {page + 1} completed.")
        page += 1

    cursor.close()
    wd.quit()

# Establish MySQL connection
mydb = connect_to_mysql()

# Example usage
wd = webdriver.Chrome()
cursor = mydb.cursor()
indeed_scraper('https://www.indeed.com/jobs?q=sustainability&l=United+States', 0, wd, cursor)


AttributeError: 'NoneType' object has no attribute 'text'

In [45]:
import mysql.connector
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Function to connect to MySQL database
def connect_to_mysql():
    return mysql.connector.connect(
        host='database-1.cux1s0fa60hj.us-east-2.rds.amazonaws.com',
        database='job_scrape',
        user='admin',
        password='Lenovo2002'
    )

def indeed_scraper(webpage, page_number, wd, cursor):  
    max_pages = 5  # Set a maximum number of pages to scrape
    page = 0

    while page < max_pages:
        url = f"{webpage}&start={page * 10}"
        wd.get(url)

        try:
            WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.job_seen_beacon')))
        except TimeoutException:
            print("Timed out waiting for page to load")
            break

        soup = BeautifulSoup(wd.page_source, 'html.parser')
        jobs = soup.find_all('div', class_='job_seen_beacon')

        if not jobs:
            print("No more jobs found.")
            break

        for job in jobs:
            data = {}
            try:
                job_id = job['data-jk']
            except KeyError:
                job_id = "Not listed"

            # Example: Extracting job details
            job_title = job.find('h2', class_='jobTitle').text.strip()
            company_name = job.find('span', class_='company').text.strip()
            location = job.find('span', class_='location').text.strip()
            job_link = 'https://www.indeed.com' + job.find('a')['href']
            skill = job.find('div', class_='jobsearch-SkillsWrapper').text.strip()
            # Adjust salary extraction logic based on your specific case
            salary = "Not listed"  
            description = "Not listed"  # You can add description extraction logic here

            # Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO job_listings (job_id, title, company, location, skill, job_link, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, skill, job_link, description)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
            
        print(f"Page {page + 1} completed.")
        page += 1

    cursor.close()
    wd.quit()

# Establish MySQL connection
mydb = connect_to_mysql()

# Example usage
wd = webdriver.Chrome()
cursor = mydb.cursor()
indeed_scraper('https://www.indeed.com/jobs?q=sustainability&l=United+States', 0, wd, cursor)


AttributeError: 'NoneType' object has no attribute 'text'

In [40]:
def indeed_scraper(webpage, page_number, wd, cursor):
    next_page = webpage + str(page_number)
    print(str(next_page))
    wd.get(next_page)

    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("div", class_="job_seen_beacon")

    for job in jobs:
        job_head_elem = job.find("h2", class_="jobTitle")
        # JobId
        job_id = job_head_elem.find("span")["id"].split("-")[1]

        # Check if the JobId already exists in the table
        select_query = "SELECT job_id FROM job_listings WHERE job_id = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()

        if result is None:
            # Job Title
            job_title = job_head_elem.find("span").text.strip()

            # Job Link
            job_link = "https://www.indeed.com" + job_head_elem.find("a")["href"]

            # Company Name
            company_name = job.find("span", class_="companyName").text.strip()

            # Location
            location = job.find("div", class_="companyLocation").text.strip()

            wd.get(job_link)
            time.sleep(3)
            job_src = wd.page_source
            job_soup = bs(job_src, "html.parser")

            # Salary
            salary_elem = None
            for elem in job_soup.find_all("div", class_="css-tvvxwd ecydgvn1"):
                text = elem.text.strip()
                if any(char.isdigit() for char in text):
                    salary_elem = elem
                    break
            salary = salary_elem.text.strip() if salary_elem else ""

            # Job Description
            job_des_elem = job_soup.find("div", id="jobDescriptionText")
            job_des = job_des_elem.text.strip() if job_des_elem else ""

            # Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO job_listings (job_id, title, company, location, skill, job_link, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
        else:
            print("Skipping duplicate JobId:", job_id)
    print("Data updated")

    if page_number < 1000:
        page_number += 10
        indeed_scraper(webpage, page_number, wd, cursor)
    else:
        cursor.close()
        wd.quit()

# Example usage
wd = webdriver.Chrome()
cursor = mydb.cursor()
indeed_scraper('https://www.indeed.com/jobs?q=sustainability&l=United+States', 0, wd, cursor)


https://www.indeed.com/jobs?q=sustainability&l=United+States0


AttributeError: 'NoneType' object has no attribute 'text'

In [41]:
def indeed_scraper(webpage, page_number, wd, cursor):
    next_page = webpage + str(page_number)
    print(str(next_page))
    wd.get(next_page)

    jobs_src = wd.page_source
    soup = bs(jobs_src, "html.parser")
    jobs = soup.find_all("div", class_="job_seen_beacon")

    for job in jobs:
        job_head_elem = job.find("h2", class_="jobTitle")
        # JobId
        job_id = job_head_elem.find("span")["id"].split("-")[1]

        # Check if the JobId already exists in the table
        select_query = "SELECT job_id FROM job_listings WHERE job_id = %s"
        cursor.execute(select_query, (job_id,))
        result = cursor.fetchone()

        if result is None:
            # Job Title
            job_title = job_head_elem.find("span").text.strip()

            # Job Link
            job_link = "https://www.indeed.com" + job_head_elem.find("a")["href"]

            # Company Name
            company_name_elem = job.find("span", class_="companyName")
            company_name = company_name_elem.text.strip() if company_name_elem else "Not listed"

            # Location
            location_elem = job.find("div", class_="companyLocation")
            location = location_elem.text.strip() if location_elem else "Not listed"

            wd.get(job_link)
            time.sleep(3)
            job_src = wd.page_source
            job_soup = bs(job_src, "html.parser")

            # Salary
            salary_elem = None
            for elem in job_soup.find_all("div", class_="css-tvvxwd ecydgvn1"):
                text = elem.text.strip()
                if any(char.isdigit() for char in text):
                    salary_elem = elem
                    break
            salary = salary_elem.text.strip() if salary_elem else ""

            # Job Description
            job_des_elem = job_soup.find("div", id="jobDescriptionText")
            job_des = job_des_elem.text.strip() if job_des_elem else ""

            # Insert the data into the MySQL table
            insert_query = '''
                INSERT INTO job_listings (job_id, title, company, location, skill, job_link, description)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            '''
            insert_values = (job_id, job_title, company_name, location, job_link, salary, job_des)
            cursor.execute(insert_query, insert_values)
            mydb.commit()
        else:
            print("Skipping duplicate JobId:", job_id)
    print("Data updated")

    if page_number < 1000:
        page_number += 10
        indeed_scraper(webpage, page_number, wd, cursor)
    else:
        cursor.close()
        wd.quit()

# Example usage
wd = webdriver.Chrome()
cursor = mydb.cursor()
indeed_scraper('https://www.indeed.com/jobs?q=sustainability&l=United+States', 0, wd, cursor)


https://www.indeed.com/jobs?q=sustainability&l=United+States0
Skipping duplicate JobId: 4c85956d2c2ade13
Skipping duplicate JobId: d98589c0592f05a8
Skipping duplicate JobId: ec8dc41a095acff6
Skipping duplicate JobId: 0b33caade9b47991
Data updated
https://www.indeed.com/jobs?q=sustainability&l=United+States10
Skipping duplicate JobId: 98f2b0e10eed10f1
Skipping duplicate JobId: f356228f17ae0bd5
Skipping duplicate JobId: 0927f21c268c4ff4
Skipping duplicate JobId: 93d9574df6dcb5c7
Skipping duplicate JobId: 1c43d40853b4599f
Skipping duplicate JobId: 4c85956d2c2ade13
Skipping duplicate JobId: 9c40dd7744a52283
Skipping duplicate JobId: 7d36b2d0b008f42c
Skipping duplicate JobId: 4c047632c2b4aac5
Skipping duplicate JobId: d98589c0592f05a8
Skipping duplicate JobId: 43d2871359e0357e
Skipping duplicate JobId: 9f409082ae0d4082
Skipping duplicate JobId: 999be210d521ac4f
Skipping duplicate JobId: a3aacb4bc365fe23
Data updated
https://www.indeed.com/jobs?q=sustainability&l=United+States20
Skipping dup